In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
input_size = 784  # 28*28
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001   # step size

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



We define our neural network as class `NeuralNet`. It inherits from `nn.Module`, which is a base class for all neural network modules in PyTorch.

*   The `__init__()` method initializes the neural network layers.
*   `self.fc1` is the first fully connected (linear) layer.
*   `self.relu` is an activation layer using the Rectified Linear Unit (ReLU) function.
*   `self.fc2` is the second fully connected layer.

The `forward()` method defines how the input `x` flows through the network layers. First, the input is passed through the `fc1` layer, then the `ReLU` activation function is applied to the output of `fc1` to introduce non-linearity, and finally, the result passes through the `fc2` layer.




In [ ]:
# Define a fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    # define the layers
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)   # 784 dimension to 500 dimension
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  # 500 dimension to 10 dimension

    # tell the model how to process input sample x
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# instantiate the model
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

*  `nn.CrossEntropyLoss()` is used as the loss function, suitable for multi-class classification problems. This function combines `nn.LogSoftmax()` and `nn.NLLLoss()` (Negative Log Likelihood Loss) in one single class. It calculates the loss between the model's output and the actual labels, encouraging the output to match the labels.

*  `torch.optim.Adam(model.parameters(), lr=learning_rate)` initializes the Adam optimizer, which adjusts the model's parameters (weights and biases) based on the learning rate and the gradients of the loss function with respect to the parameters.

*  In the training loop, we clears old gradients (`optimizer.zero_grad()`), performs backpropagation (`loss.backward()`) to compute the gradients, and updates the model parameters (`optimizer.step()`).

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()   # define the loss computation method (not computed yet!)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)    # put model parameters into the optimizer; lr is "learning rate", which is the gradient descent "step size"

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels) # Compute the difference between prediction and label as the loss (i.e., "objective" or "cost")

        # Backward and optimize
        optimizer.zero_grad()   # important!!!
        loss.backward()   # compute gradient
        optimizer.step()  # update parameters with gradient descent

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2890
Epoch [1/5], Step [200/600], Loss: 0.3048
Epoch [1/5], Step [300/600], Loss: 0.1530
Epoch [1/5], Step [400/600], Loss: 0.1874
Epoch [1/5], Step [500/600], Loss: 0.1634
Epoch [1/5], Step [600/600], Loss: 0.1737
Epoch [2/5], Step [100/600], Loss: 0.0324
Epoch [2/5], Step [200/600], Loss: 0.0785
Epoch [2/5], Step [300/600], Loss: 0.0763
Epoch [2/5], Step [400/600], Loss: 0.1230
Epoch [2/5], Step [500/600], Loss: 0.0708
Epoch [2/5], Step [600/600], Loss: 0.0726
Epoch [3/5], Step [100/600], Loss: 0.0765
Epoch [3/5], Step [200/600], Loss: 0.1053
Epoch [3/5], Step [300/600], Loss: 0.0423
Epoch [3/5], Step [400/600], Loss: 0.0793
Epoch [3/5], Step [500/600], Loss: 0.0321
Epoch [3/5], Step [600/600], Loss: 0.0919
Epoch [4/5], Step [100/600], Loss: 0.0523
Epoch [4/5], Step [200/600], Loss: 0.0464
Epoch [4/5], Step [300/600], Loss: 0.0658
Epoch [4/5], Step [400/600], Loss: 0.0328
Epoch [4/5], Step [500/600], Loss: 0.0436
Epoch [4/5], Step [600/600], Loss:

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the network on the 10000 test images: 97.63 %
